In [ ]:
# Import necessary libraries
import boto3, re, sys, math, json, os
import numpy as np 
import io as io
import pandas as pd 

In [ ]:
!pip install python-dotenv
%reload_ext dotenv
%dotenv

In [ ]:
# Access and Credential details
bucket_name = os.environ['BUCKET_NAME'] # <--- Environment variable for Bucket Name
ecs_access_key_id=os.environ['ECS_ACCESS_KEY_ID']  # <--- Environment variable for ECS Access Key
ecs_secret_access_key=os.environ['ECS_SECRET_ACCESS_KEY'] # <--- Environment variable for Secret Access Key
endpoint_url=os.environ['ENDPOINT_URL'] # <--- Environment variable for Endpoint URL

In [ ]:
s3 = boto3.resource(service_name='s3',aws_access_key_id=ecs_access_key_id,aws_secret_access_key=ecs_secret_access_key,endpoint_url=endpoint_url)

for bucket in s3.buckets.all():
  print(bucket.name)

In [ ]:
s3_client = boto3.client(service_name='s3',aws_access_key_id=ecs_access_key_id,aws_secret_access_key=ecs_secret_access_key,endpoint_url=endpoint_url)

from_aws = "sagemaker-created-me"

evaluate=s3_client.get_object(Bucket=from_aws, Key = 'bank_clean.csv')

In [ ]:
try: 
    model_data = pd.read_csv(io.BytesIO(evaluate['Body'].read()),index_col=0) 
    print('Success: Data loaded into dataframe.') 
except Exception as e: 
    print('Data load error: ',e)

In [ ]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))]) 
print(train_data.shape, test_data.shape)

In [ ]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

In [ ]:
# Upload file to ECS bucket with "upload_" prefix 
s3=boto3.resource(service_name='s3',aws_access_key_id=ecs_access_key_id,aws_secret_access_key=ecs_secret_access_key,endpoint_url=endpoint_url) 
upload_train=s3.Bucket(bucket_name).Object(os.path.join('train/azure_train.csv')).upload_file('train.csv')

In [ ]:
download_train=s3.Bucket(bucket_name).Object(os.path.join('train/azure_train.csv')).download_file('download_train.csv') 
# Check in the terminal for downloaded file 
# You should be able to see download_train.csv and train.csv